In [3]:
from huggingface_hub import login
login()

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
import re

# Load the Mixtral model and tokenizer
model_name = "mistralai/Mistral-7B-v0.1"  # Replace with the appropriate model name or path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define the prompt template for extracting structured data as JSON
prompt_template = """
You are an AI assistant capable of extracting structured information from medical documents. Given a passage, extract specific information and return it only as a JSON object. The JSON should include the following fields depending on the type of document:

Medical Tests (анализы):
Test Name 
Reference Range (e.g., norms for the test)
Units (e.g., г/дл, %)
Test Results (e.g., numerical values or text)
Test Date
Test Location (e.g., the name of the medical institution)
Test Address
Medical Studies (исследования):
Study Name (e.g., "Ультразвуковое исследование")
Study Date
Study Location (e.g., the name of the medical institution)
Device (e.g., the equipment used for the study)
Study Protocol
Study Conclusion
Study Recommendations
Study Address
If certain information is not present in the text, return those fields as 'null'. The extracted data should be structured and presented as a json file to the user.

Now, process the following text: {input_text}
"""

def extract_json_from_text(input_text):
    # Format the prompt with the input text
    prompt = prompt_template.format(input_text=input_text)
    
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    
    # Generate the completion using Mixtral
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=1500,  # Adjust length based on expected output size
            num_beams=5,      # Adjust number of beams for beam search
            early_stopping=True
        )
    
    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract and return the JSON part from the response
    json_match = re.search(r'\{[\w\W]*\}', generated_text)
    if json_match:
        return json.loads(json_match.group())
    else:
        return {"error": "No valid JSON found in the response"}

# Example usage
input_text = """
РЕЗУЛЬТАТЫ ЛАБОРАТОРНЫХ ИССЛЕДОВАНИЙ
ФИО пациента:
Шахмурадян Михаил Андреевич
Вид биоматериала: Венозная кровь
Тип пробы:
Дата рождения:
07.10.1993
Плазма крови
Дата поступления
биоматериала:
14.07.2024 14:26
Мужской
Пол:
Дата выполнения
исследования:
15.07.2024 12:32
Исследование выполнено на автоматическом биохимическом анализаторе AU480 (Beckman Coulter, США)
№
Исследование
Результат
Ед. изм.
Интерпретация
БИОХИМИЧЕСКИЕ ИССЛЕДОВАНИЯ КРОВИ
134
ммоль/л
4.76
Глюкоза
4,39
5,93
156
мкмоль/л
23.1
Железо
12,5
32,2
160
нг/мл
115.0
Ферритин
20,0
250,0
Результат лабораторного исследования не является диагнозом. Согласно федеральному закону №323-ФЗ от 21.11.2011 (ред. от
29.07.2017) «Об основах охраны здоровья граждан в Российской Федерации» диагноз устанавливает лечащий врач, основываясь
на всестороннем обследовании пациента: сбор и анализ жалоб, данные анамнеза и осмотра, проведение лабораторных,
инструментальных и иных исследований.
Выполнил:
Биолог КДЛ Елисеева Л. В.
1 из 1
 B = H Ht) Tt. 8-928-111-22-11
_] e-mail: official@ avenumed.ru
eb NABOPATOPIA WwWw.avenumed.ru

000 “CeBepnan 3Be3ga” WHH 6141045157 OFPH 1136181005135 Siny. J1041-01050-61/00327726"""

extracted_data = extract_json_from_text(input_text)
print(json.dumps(extracted_data, indent=4, ensure_ascii=False))


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

KeyboardInterrupt: 